In [189]:
import tensorflow as tf
import numpy as np
import os
import time

In [190]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [191]:
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [192]:
# 文本中的非重复字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [193]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [194]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [195]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [196]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [197]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [198]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [199]:
# 由于后面的GRU层设定了stateful=True, 所以必须事先指定batch_size，
# 对于Sequential产生的模型，使用参数batch_input_shape=[batch_size, None]来指定
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]
                             ),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [200]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [201]:
a = next(iter(dataset.take(1)))

In [111]:
model.layers[1](model.layers[0](a[0])).shape

TensorShape([64, 100, 1024])

In [112]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [113]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [114]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173464


In [115]:
model.compile(optimizer='adam', loss=loss)

In [116]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [117]:
EPOCHS=6

In [118]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], steps_per_epoch=50)

Train for 50 steps
Epoch 1/6
50/50 [==============================] - 90s 2s/step - loss: 3.3997
Epoch 2/6
50/50 [==============================] - 99s 2s/step - loss: 2.4831
Epoch 3/6
50/50 [==============================] - 95s 2s/step - loss: 2.2691
Epoch 4/6
22/50 [============>.................] - ETA: 52s - loss: 2.1800

W0217 14:56:10.183674 4628829632 training_v2.py:146] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 300 batches). You may need to use the repeat() function when building your dataset.


22/50 [============>.................] - ETA: 52s - loss: 2.1800

In [120]:
tf.train.latest_checkpoint('./training_checkpoints/')

'./training_checkpoints/ckpt_4'

In [121]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [165]:
start_string=u"ROMEO: "

In [166]:
num_generate = 1000

In [167]:
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

In [168]:
text_generated = []
temperature = 1.0
model.reset_states()

In [169]:
predictions = model(input_eval)
predictions = tf.squeeze(predictions, 0)
predictions = predictions / temperature
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

In [170]:
input_eval = tf.expand_dims([predicted_id], 0)

In [171]:
input_eval

<tf.Tensor: id=87181, shape=(1, 1), dtype=int32, numpy=array([[39]], dtype=int32)>

In [172]:
predicted_id

39

In [173]:
input_eval = tf.expand_dims([predicted_id], 0)

In [174]:
input_eval

<tf.Tensor: id=87184, shape=(1, 1), dtype=int32, numpy=array([[39]], dtype=int32)>

In [175]:
text_generated.append(idx2char[predicted_id])

In [176]:
text_generated

['a']

In [178]:
predictions = model(input_eval)

In [179]:
predictions = tf.squeeze(predictions, 0)

In [180]:
predictions = predictions / temperature

In [181]:
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

In [183]:
input_eval = tf.expand_dims([predicted_id], 0)

In [184]:
text_generated.append(idx2char[predicted_id])

In [185]:
text_generated

['a', 'd']